# Google Calendar Event Fetcher

#### The Notebook demonstrates how to programmatically retrieve and process Google Calendar events for a given user and date range.

##### Key Steps:

Authentication: Load user credentials from a token file.

API Call: Fetch events between specified start/end dates using the Google Calendar API.

Data Processing: Extract event details (start/end times, attendees) and structure them into a clean format.

Output: Return a list of events with attendee counts and time slots.
```

### Importing Required Libraries

In [8]:
import json
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

### Defining the retrive_calendar_events Function that includes : 
#### 1. Authenticating with Google Calendar
#### 2. Fetching Events
#### 3. Processing Events & Structuring

In [12]:
def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "./Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"]})
    return events_list

In [29]:
import json
from datetime import datetime
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "./Keys/" + user.split("@")[0] + ".token"
    try:
        user_creds = Credentials.from_authorized_user_file(token_path)
        calendar_service = build("calendar", "v3", credentials=user_creds)
        events_result = calendar_service.events().list(
            calendarId='primary',
            timeMin=start,
            timeMax=end,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])
        
        for event in events:
            attendee_list = []
            try:
                for attendee in event.get("attendees", []):
                    attendee_list.append(attendee['email'])
            except:
                attendee_list.append("SELF")
            start_time = event["start"]["dateTime"]
            end_time = event["end"]["dateTime"]
            events_list.append({
                "StartTime": start_time,
                "EndTime": end_time,
                "NumAttendees": len(set(attendee_list)),
                "Attendees": list(set(attendee_list)),
                "Summary": event.get("summary", "")
            })
    except FileNotFoundError:
        events_list.append({"error": f"Token file not found for {user}"})
    except Exception as e:
        events_list.append({"error": f"Failed to retrieve events for {user}: {str(e)}"})
    return events_list

def process_calendar_events(json_input, start_time, end_time):
    # Parse JSON input
    data = json.loads(json_input) if isinstance(json_input, str) else json_input
    
    # Extract relevant fields
    from_email = data.get("From", "")
    attendees = data.get("Attendees", [])
    
    # Convert start_time and end_time to RFC 3339 format
    try:
        start_dt = datetime.strptime(start_time, "%d-%m-%YT%H:%M:%S")
        end_dt = datetime.strptime(end_time, "%d-%m-%YT%H:%M:%S")
        start_time_rfc3339 = start_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
        end_time_rfc3339 = end_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    except ValueError:
        return {"error": "Invalid datetime format. Expected DD-MM-YYYYTHH:MM:SS"}
    
    # Extract emails and usernames
    emails = [from_email] if from_email else []
    usernames = [from_email.split("@")[0]] if from_email else []
    for attendee in attendees:
        email = attendee.get("email", "")
        if email:
            emails.append(email)
            usernames.append(email.split("@")[0])
    emails = list(set(emails))  # Remove duplicate emails
    usernames = list(set(usernames))  # Remove duplicate usernames
    
    # Retrieve events for each user
    result = {}
    for email, username in zip(emails, usernames):
        events = retrive_calendar_events(email, start_time_rfc3339, end_time_rfc3339)
        result[username] = events
        valid_events = [event for event in events if "error" not in event]
        result[username] = valid_events
    
    return result

### Calling ```retrive_calendar_events``` with Auth Token, Start Date & End Date 
#### Date Format : YYYY-MM-DD (T) HH:MM:SS +TIMEZONE (+5:30 Indicates IST Time Zone Asia/Kolkata ) 

In [34]:
input1 ={
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

In [36]:
event1 = process_calendar_events(input,'09-07-2025T12:34:55','23-07-2025T12:34:55')

### Output that describing sample event : 

In [37]:
event1

{'userthree.amd': [{'StartTime': '2025-07-09T16:00:00+05:30',
   'EndTime': '2025-07-10T07:30:00+05:30',
   'NumAttendees': 0,
   'Attendees': [],
   'Summary': 'Off Hours'},
  {'StartTime': '2025-07-10T16:00:00+05:30',
   'EndTime': '2025-07-11T07:30:00+05:30',
   'NumAttendees': 0,
   'Attendees': [],
   'Summary': 'Off Hours'},
  {'StartTime': '2025-07-11T16:00:00+05:30',
   'EndTime': '2025-07-12T07:30:00+05:30',
   'NumAttendees': 0,
   'Attendees': [],
   'Summary': 'Off Hours'}],
 'userone.amd': [{'StartTime': '2025-07-09T16:00:00+05:30',
   'EndTime': '2025-07-10T07:30:00+05:30',
   'NumAttendees': 0,
   'Attendees': [],
   'Summary': 'Off Hours'},
  {'StartTime': '2025-07-10T16:00:00+05:30',
   'EndTime': '2025-07-11T07:30:00+05:30',
   'NumAttendees': 0,
   'Attendees': [],
   'Summary': 'Off Hours'},
  {'StartTime': '2025-07-11T16:00:00+05:30',
   'EndTime': '2025-07-12T07:30:00+05:30',
   'NumAttendees': 0,
   'Attendees': [],
   'Summary': 'Off Hours'}],
 'usertwo.amd': [{

In [39]:
import json
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "./Keys/" + user.split("@")[0] + ".token"
    try:
        user_creds = Credentials.from_authorized_user_file(token_path)
        calendar_service = build("calendar", "v3", credentials=user_creds)
        events_result = calendar_service.events().list(
            calendarId='primary',
            timeMin=start,
            timeMax=end,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])
        
        for event in events:
            attendee_list = []
            try:
                for attendee in event.get("attendees", []):
                    attendee_list.append(attendee['email'])
            except:
                attendee_list.append("SELF")
            # Handle both dateTime and date fields
            start_time = event["start"].get("dateTime", event["start"].get("date", ""))
            end_time = event["end"].get("dateTime", event["end"].get("date", ""))
            if not start_time or not end_time:
                continue  # Skip events with invalid time formats
            events_list.append({
                "StartTime": start_time,
                "EndTime": end_time,
                "NumAttendees": len(set(attendee_list)),
                "Attendees": list(set(attendee_list)),
                "Summary": event.get("summary", "")
            })
    except FileNotFoundError:
        events_list.append({"error": f"Token file not found for {user}"})
    except Exception as e:
        events_list.append({"error": f"Failed to retrieve events for {user}: {str(e)}"})
    return events_list

def populate_events_in_json(json_input):
    # Parse JSON input
    data = json.loads(json_input) if isinstance(json_input, str) else json_input.copy()
    
    # Extract Datetime and compute time range
    start_time_str = data.get("Datetime", "")
    try:
        start_dt = datetime.strptime(start_time_str, "%d-%m-%YT%H:%M:%S")
        start_time_rfc3339 = start_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
        end_dt = start_dt + timedelta(days=14)
        end_time_rfc3339 = end_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    except ValueError:
        return {"error": "Invalid datetime format. Expected DD-MM-YYYYTHH:MM:SS"}
    
    # Extract Attendees
    attendees = data.get("Attendees", [])
    
    # Update events for each attendee
    for attendee in attendees:
        email = attendee.get("email", "")
        if not email:
            attendee["events"] = []
            continue
        # Fetch events for the user
        events = retrive_calendar_events(email, start_time_rfc3339, end_time_rfc3339)
        # Filter out error entries
        valid_events = [event for event in events if "error" not in event]
        attendee["events"] = valid_events
    
    return data

In [41]:
populate_events_in_json(input1)

{'Request_id': '6118b54f-907b-4451-8d48-dd13d76033a5',
 'Datetime': '09-07-2025T12:34:55',
 'Location': 'IIT Mumbai',
 'From': 'userone.amd@gmail.com',
 'Attendees': [{'email': 'usertwo.amd@gmail.com',
   'events': [{'StartTime': '2025-07-09T18:00:00+05:30',
     'EndTime': '2025-07-10T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-10T18:00:00+05:30',
     'EndTime': '2025-07-11T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-11T18:00:00+05:30',
     'EndTime': '2025-07-12T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-12',
     'EndTime': '2025-07-13',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Weekend'},
    {'StartTime': '2025-07-12T18:00:00+05:30',
     'EndTime': '2025-07-13T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summ